In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
import nltk
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
nltk.download("stopwords")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
df=pd.read_csv("/kaggle/input/real-or-fake-fake-jobposting-prediction/fake_job_postings.csv",nrows=5000)
df=df.fillna("nil")

In [3]:
# df["salary_range"].unique()

In [4]:
# impute=SimpleImputer()
df["salary_range"]=df["salary_range"].replace("nil",'0')

In [5]:
df

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,0,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,nil,0,1,0,Other,Internship,nil,nil,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,0,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,nil,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",nil,0,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,nil,0,1,0,nil,nil,nil,nil,nil,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,0,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",nil,0,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4996,Operations Support,"PH, , Quezon City",nil,0,nil,Operations SupportCollege Graduate or at least...,Operations SupportCollege Graduate or at least...,"Collabera is a fast growing, end-to-end inform...",0,0,0,nil,nil,nil,nil,nil,0
4996,4997,Customer Service Associate - Part Time,"US, NY, New York",nil,0,"Novitex Enterprise Solutions, formerly Pitney ...",The Customer Service Associate will be based ...,Position Qualifications: Minimum of six months...,nil,0,1,0,Part-time,Entry level,High School or equivalent,Financial Services,Customer Service,0
4997,4998,System Engineer - Neltog,nil,nil,0,Massive Media is the social media company behi...,FunctionThe IT Services-department serves our ...,nil,nil,0,1,0,nil,nil,nil,nil,nil,0
4998,4999,Product Manager,"US, NY, New York",nil,0,Recombine provides clinical genetic testing. C...,Recombine seeks to develop the most comprehens...,"About You• Master’s or PhD in Genetics, Geneti...",Participate and contribute to an environment w...,0,1,0,Full-time,nil,nil,nil,Product Management,0


In [6]:
df.sample()

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
3468,3469,Graphic Designer,"US, MA,",nil,0,At TeeSmile we make it easy to raise money wit...,Do you have an eye for design and automaticall...,Proficient in Adobe Illustrator and Photoshop....,nil,1,1,1,Contract,Not Applicable,nil,nil,Design,0


In [7]:
df.isnull().sum()

job_id                 0
title                  0
location               0
department             0
salary_range           0
company_profile        0
description            0
requirements           0
benefits               0
telecommuting          0
has_company_logo       0
has_questions          0
employment_type        0
required_experience    0
required_education     0
industry               0
function               0
fraudulent             0
dtype: int64

In [8]:
df.dtypes

job_id                  int64
title                  object
location               object
department             object
salary_range           object
company_profile        object
description            object
requirements           object
benefits               object
telecommuting           int64
has_company_logo        int64
has_questions           int64
employment_type        object
required_experience    object
required_education     object
industry               object
function               object
fraudulent              int64
dtype: object

In [9]:
one_hot=OneHotEncoder(sparse_output=False)
one=one_hot.fit_transform(df[["title","salary_range","location","department","employment_type","required_experience","required_education","industry","function"]])
one=pd.DataFrame(one,columns=one_hot.get_feature_names_out())

In [10]:
one

,title_ Piping Material Engineer,"title_ Discipline Manager Civil, Structural, Marine, Architectural",title_ ServiceNow Consultant,title_ Account Executive/Digital Ad Sales,title_ Agile Business Analyst,title_ CAD Operator,title_ CARER/SENIOR CARER POSITIONS,title_ Database Development Lead,title_ Downtown Valet Attendant,title_ Executive Assistant / Office Manager,...,function_Purchasing,function_Quality Assurance,function_Research,function_Sales,function_Science,function_Strategy/Planning,function_Supply Chain,function_Training,function_Writing/Editing,function_nil
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
def text_preprocessing(text):
    data=text.lower()
    data_re=re.sub(r'\W+',' ',data)
    data_tokenize=word_tokenize(data_re)
    tokens=[word for word in data_tokenize if word not in stopwords.words("english")]
    return " ".join(tokens)

In [12]:
def preprocess_columns(df,col):
    for cl in col:
        df[cl]=df[cl].apply(text_preprocessing)
    return df

In [13]:
def vectorizer_text(df,col):
    vectorizer=TfidfVectorizer()
    vectorizer_data=[]
    df = preprocess_columns(df, col)

    for cl in col:
#         df[cl]=df[cl].apply(preprocess_columns)
        transformed=vectorizer.fit_transform(df[cl])
        transformed_df=pd.DataFrame(transformed.toarray(),columns=vectorizer.get_feature_names_out())
        vectorizer_data.append(transformed_df)
    final_data=pd.concat(vectorizer_data,axis=1)
    return final_data

In [14]:
col = ["title", "company_profile", "description", "requirements", "benefits"]
vectorized_df = vectorizer_text(df, col)

In [15]:
vectorized_df

,000,0tz,10,100,100k,1099,12hr,130,14,15,...,όλο,όλοι,όλους,όλων,όμιλο,όμορφο,όπως,όραμα,ότι,ότου
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
concaty=pd.concat([df,one,vectorized_df],axis=1).drop(columns=["title","salary_range","location","department","employment_type","required_experience","required_education","industry","function","title", "company_profile", "description", "requirements", "benefits","fraudulent"])
concaty

,job_id,telecommuting,has_company_logo,has_questions,title_ Piping Material Engineer,"title_ Discipline Manager Civil, Structural, Marine, Architectural",title_ ServiceNow Consultant,title_ Account Executive/Digital Ad Sales,title_ Agile Business Analyst,title_ CAD Operator,...,όλο,όλοι,όλους,όλων,όμιλο,όμορφο,όπως,όραμα,ότι,ότου
0,1,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0,1,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4996,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4996,4997,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4997,4998,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4998,4999,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
X_train,X_test,Y_train,Y_test=train_test_split(concaty,df["fraudulent"],test_size=0.2,random_state=42)

In [18]:
random_classif=RandomForestClassifier()
random_classif.fit(X_train,Y_train)
random_classif.score(X_test,Y_test)

0.983